In [1]:
from pyqubo import Binary, Constraint, SubH
from pyqubo import UnaryEncInteger,LogEncInteger
import numpy as np
import sys
from qiskit_optimization.algorithms import CplexOptimizer, GurobiOptimizer
from qiskit_optimization.problems import QuadraticProgram
import csv

In [2]:
def csv_to_matrix(filename, size):
    num_rows, num_cols = size
    matrix = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            
            if len(row) != num_cols:
                raise ValueError("Number of columns in row does not match expected number of columns")
            matrix_row = []
            for value in row:
                try:
                    matrix_row.append(float(value))
                except ValueError:
                    matrix_row.append(value)
            matrix.append(matrix_row)
    if len(matrix) != num_rows:
        raise ValueError("Number of rows in matrix does not match expected number of rows")
    return matrix

In [3]:
# input the w weighting

num_of_position, num_of_mol = 5,11

# nearest neighbor interaction

w = csv_to_matrix('0117data_one_functional group.csv', (5,11))

W10 = csv_to_matrix('0117data_R1R2_double_functional group.csv', (11,11))
W34 = csv_to_matrix('0117data_R1R2_double_functional group.csv', (11,11))

W21 = csv_to_matrix('0117data_R2R3_double_functional group.csv', (11,11))
W23 = csv_to_matrix('0117data_R2R3_double_functional group.csv', (11,11))
# second nearest neighbor interaction
W04 = csv_to_matrix('0117data_R1R5_double_functional group.csv', (11,11))

W20 = csv_to_matrix('0117data_R1R3_double_functional group.csv', (11,11))
W24 = csv_to_matrix('0117data_R1R3_double_functional group.csv', (11,11))

W31 = csv_to_matrix('0117data_R2R4_double_functional group.csv', (11,11))

W30 = csv_to_matrix('0117data_R1R4_double_functional group.csv', (11,11))
W14 = csv_to_matrix('0117data_R1R4_double_functional group.csv', (11,11))

In [4]:
# create the binary variables
def create_x_vars(num_of_position, num_of_mol):
    a = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    for i in range(num_of_position):
        for j in range(num_of_mol):
            vars_name = 'x'+str(i)+'_'+str(j)
            a[i][j] = Binary(vars_name)
    return a

In [5]:
def create_x_vars_name(num_of_position, num_of_mol):
    a0 = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    a1 = np.ndarray(shape=(num_of_position, num_of_mol), dtype=Binary)
    for i in range(num_of_position):
        for j in range(num_of_mol):
            vars_name = 'x'+str(i)+'_'+str(j)
            a1[i][j] = vars_name
    return a1

In [6]:
# construct objective function
x = create_x_vars(num_of_position, num_of_mol)
summ = 0
for i in range(num_of_position):
    for j in range(num_of_mol):
        summ += w[i][j]*x[i][j]
        

summ_10 = 0
summ_21 = 0
summ_23 = 0
summ_34 = 0
summ_04 = 0

summ_20 = 0
summ_24 = 0

summ_31 = 0
summ_30 = 0
summ_14 = 0


for i in range(1,num_of_mol):
    for j in range(1,num_of_mol):
        # nearest neighbor interaction
        summ_10 += W10[j][i]*x[1][j]*x[0][i]
        summ_34 += W34[j][i]*x[3][j]*x[4][i] 

        summ_21 += W21[j][i]*x[2][j]*x[1][i]  
        summ_23 += W23[j][i]*x[2][j]*x[3][i]

        summ_04 += W04[j][i]*x[0][j]*x[4][i] 

        # second nearest neighbor interaction
        summ_20 += W20[j][i]*x[2][j]*x[0][i] 
        summ_24 += W24[j][i]*x[2][j]*x[4][i]

        summ_31 += W31[j][i]*x[3][j]*x[1][i] 
        summ_30 += W30[j][i]*x[3][j]*x[0][i] 
        summ_14 += W14[j][i]*x[1][j]*x[4][i]

obj = (summ_10 + summ_34 + summ_21 + summ_23 + summ_04 + summ_20 + summ_24 + summ_31 + summ_30 + summ_14 ) + summ  + 87.5

In [7]:
H = obj
H_model = H.compile()
H_qubo, offset = H_model.to_qubo()
print(offset)

87.5


In [8]:
def conver_pyqubo_to_docplex(qubo):
# qubo should be the to_qubo format from pyqubo
# 'check the input data correctness'
    if type(qubo) != dict:
        sys.exit("Format of input data incorrect")
    else:
# because docplex deal with the qradractic and linear terms separately, we use the loop to save these two information into two dictionary, linear and quadactic
        linear = {}
        quadratic = {}

        for i in qubo.keys():
            if i[0] == i[1]:
                linear[str(i[0])] = qubo[ (str(i[0]), str(i[0])) ]
            else:
                quadratic[i] = qubo[i]
    return linear, quadratic

In [9]:
linear, quadratic = conver_pyqubo_to_docplex(H_model.to_qubo()[0])

In [10]:
(num_of_position, num_of_mol) = (5,11)
qp = QuadraticProgram()
for i in range(len(create_x_vars_name(num_of_position, num_of_mol))):
    for j in range(len(create_x_vars_name(num_of_position, num_of_mol)[i])):
        qp.binary_var(str(create_x_vars_name(num_of_position, num_of_mol)[i][j]))
        

all_function_con = {'x0_1': 1, 'x0_2': 1, 'x0_3': 1, 'x0_4': 1, 'x0_5': 1, 'x0_6': 1, 'x0_7': 1, 'x0_8': 1, 'x0_9': 1, 'x0_10': 1,
                    'x1_1': 1, 'x1_2': 1, 'x1_3': 1, 'x1_4': 1, 'x1_5': 1, 'x1_6': 1, 'x1_7': 1, 'x1_8': 1, 'x1_9': 1, 'x1_10': 1,
                    'x2_1': 1, 'x2_2': 1, 'x2_3': 1, 'x2_4': 1, 'x2_5': 1, 'x2_6': 1, 'x2_7': 1, 'x2_8': 1, 'x2_9': 1, 'x2_10': 1,
                    'x3_1': 1, 'x3_2': 1, 'x3_3': 1, 'x3_4': 1, 'x3_5': 1, 'x3_6': 1, 'x3_7': 1, 'x3_8': 1, 'x3_9': 1, 'x3_10': 1,
                    'x4_1': 1, 'x4_2': 1, 'x4_3': 1, 'x4_4': 1, 'x4_5': 1, 'x4_6': 1, 'x4_7': 1, 'x4_8': 1, 'x4_9': 1, 'x4_10': 1}


linear_constr = {j:1 for j in create_x_vars_name(num_of_position, num_of_mol)[0] }
linear_constr1 = {j:1 for j in create_x_vars_name(num_of_position, num_of_mol)[1] }
linear_constr2 = {j:1 for j in create_x_vars_name(num_of_position, num_of_mol)[2] }
linear_constr3 = {j:1 for j in create_x_vars_name(num_of_position, num_of_mol)[3] }
linear_constr4 = {j:1 for j in create_x_vars_name(num_of_position, num_of_mol)[4] }


qp.linear_constraint(linear=all_function_con, sense="==", rhs=3, name="lin_eq0")
qp.linear_constraint(linear=linear_constr, sense="==", rhs=1, name="lin_eq")
qp.linear_constraint(linear=linear_constr1, sense="==", rhs=1, name="lin_eq1")
qp.linear_constraint(linear=linear_constr2, sense="==", rhs=1, name="lin_eq2")
qp.linear_constraint(linear=linear_constr3, sense="==", rhs=1, name="lin_eq3")
qp.linear_constraint(linear=linear_constr4, sense="==", rhs=1, name="lin_eq4")

qp.minimize(linear = linear, quadratic = quadratic)
print(qp.export_as_lp_string())   

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 9.525495540000 x0_1 - 11.788898860000 x0_2 + 0.540280083000 x0_3
      - 2.146060260000 x0_4 - 2.114685110000 x0_5 - 1.973918200000 x0_6
      - 3.808315707000 x0_7 - 8.994628002000 x0_8 + 1.657862926000 x0_9
      + 4.926526053000 x0_10 - 0.893564272000 x1_1 - 0.112950540000 x1_2
      + 1.000239782000 x1_3 - 0.555967658000 x1_4 + 0.283631356000 x1_5
      + 1.310853767000 x1_6 + 1.431961846000 x1_7 - 0.619345461000 x1_8
      + 2.729010547000 x1_9 + 1.770185963000 x1_10 - 5.650000000000 x2_1
      - 9.293946933000 x2_2 - 2.233910680000 x2_3 - 2.127235170000 x2_4
      - 1.885019012000 x2_5 - 5.801265235000 x2_6 - 1.002749794000 x2_7
      - 4.924643544000 x2_8 + 2.366941316000 x2_9 + 2.299170992000 x2_10
      - 0.893564272000 x3_1 - 0.112950540000 x3_2 + 1.000239782000 x3_3
      - 0.555967658000 x3_4 + 0.283631356000 x3_5 + 1.310853767000 x3_6
      + 1.431961846000 x3_7 - 0.61934

In [11]:
gurobi_result = GurobiOptimizer(disp=True).solve(qp)
print("gurobi")

Restricted license - for non-production use only - expires 2024-10-28
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 55 columns and 105 nonzeros
Model fingerprint: 0xe1fea08a
Model has 1000 quadratic objective terms
Variable types: 0 continuous, 55 integer (55 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+01]
  QObjective range [9e-03, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective -9.3146248
Presolve removed 0 rows and 5 columns
Presolve time: 0.02s
Presolved: 1006 rows, 1050 columns, 3100 nonzeros
Variable types: 0 continuous, 1050 integer (1050 binary)

Root relaxation: objective -3.795096e+01, 269 iterations, 0.01 seconds (0.01 work uni

In [12]:
optimal = []
for i in gurobi_result.variables_dict.keys():
    if gurobi_result.variables_dict[i]==1:
        optimal.append(i)

In [13]:
gurobi_result.status
print('The lowest BDE is:',gurobi_result.fval+offset, 'kcal/mol')
print('with the structure:', optimal)

The lowest BDE is: 65.63663199 kcal/mol
with the structure: ['x0_1', 'x1_0', 'x2_2', 'x3_0', 'x4_8']


**Because the symmetrization of our QUBO formulation, the optimal results are degenerate, where
['x0_1', 'x1_0', 'x2_2', 'x3_0', 'x4_8'] and ['x0_8', 'x1_0', 'x2_2', 'x3_0', 'x4_1'] are both the optimal solutions with predicted BDE=65.63663199 kcal/mol.